<a href="https://colab.research.google.com/github/onlyforthesis/114-/blob/main/20241204%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deap numpy yfinance

In [ ]:
pip install pyspark yfinance numpy pandas

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

def analyze_stock_performance(symbol="2330.TW", period="5y"):
    """分析股票表現並提供建議"""
    # 獲取股票數據
    stock = yf.Ticker(symbol)
    hist = stock.history(period=period)

    # 技術指標計算
    hist['SMA20'] = hist['Close'].rolling(window=20).mean()
    hist['SMA60'] = hist['Close'].rolling(window=60).mean()
    hist['RSI'] = calculate_rsi(hist['Close'])
    hist['MACD'], hist['Signal'] = calculate_macd(hist['Close'])

    # 波動性分析
    volatility = hist['Close'].pct_change().std() * np.sqrt(252)
    avg_volume = hist['Volume'].mean()

    # 趨勢分析
    current_trend = analyze_trend(hist)

    # 支撐與壓力位計算
    support, resistance = calculate_support_resistance(hist)

    # 風險評估
    risk_level = assess_risk(hist, volatility)

    # 交易建議生成
    recommendations = generate_recommendations(hist, current_trend, support, resistance, risk_level)

    return {
        'technical_indicators': {
            'current_price': hist['Close'][-1],
            'sma20': hist['SMA20'][-1],
            'sma60': hist['SMA60'][-1],
            'rsi': hist['RSI'][-1],
            'macd': hist['MACD'][-1]
        },
        'market_metrics': {
            'volatility': volatility,
            'avg_volume': avg_volume,
            'support': support,
            'resistance': resistance
        },
        'analysis': {
            'trend': current_trend,
            'risk_level': risk_level,
            'recommendations': recommendations
        }
    }

def calculate_rsi(prices, period=14):
    """計算RSI指標"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    """計算MACD指標"""
    exp1 = prices.ewm(span=fast, adjust=False).mean()
    exp2 = prices.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    return macd, signal_line

def analyze_trend(data):
    """分析當前趨勢"""
    last_price = data['Close'][-1]
    sma20 = data['SMA20'][-1]
    sma60 = data['SMA60'][-1]

    if last_price > sma20 > sma60:
        return "強勁上升"
    elif last_price > sma20 and sma20 < sma60:
        return "可能反轉向上"
    elif last_price < sma20 < sma60:
        return "下降趨勢"
    elif last_price < sma20 and sma20 > sma60:
        return "可能反轉向下"
    return "橫盤整理"

def calculate_support_resistance(data, window=20):
    """計算支撐與壓力位"""
    recent_data = data.tail(window)
    support = recent_data['Low'].min()
    resistance = recent_data['High'].max()
    return support, resistance

def assess_risk(data, volatility):
    """評估風險等級"""
    if volatility > 0.4:
        return "高風險"
    elif volatility > 0.2:
        return "中等風險"
    return "低風險"

def generate_recommendations(data, trend, support, resistance, risk_level):
    """生成交易建議"""
    last_price = data['Close'][-1]
    rsi = data['RSI'][-1]
    macd = data['MACD'][-1]
    signal = data['Signal'][-1]

    recommendations = []

    # 趨勢建議
    if trend in ["強勁上升", "可能反轉向上"]:
        if rsi < 70:  # 未超買
            recommendations.append("可考慮分批買入")
    elif trend in ["下降趨勢", "可能反轉向下"]:
        if rsi > 30:  # 未超賣
            recommendations.append("建議觀望或減碼")

    # 技術指標建議
    if macd > signal:
        recommendations.append("MACD顯示買入信號")
    elif macd < signal:
        recommendations.append("MACD顯示賣出信號")

    # 風險建議
    if risk_level == "高風險":
        recommendations.append("建議使用較嚴格的停損點")

    return recommendations

# 執行分析
results = analyze_stock_performance()

<ipython-input-1-c89e13a77401>:73: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  last_price = data['Close'][-1]
<ipython-input-1-c89e13a77401>:74: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sma20 = data['SMA20'][-1]
<ipython-input-1-c89e13a77401>:75: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sma60 = data['SMA60'][-1]
<ipython-input-1-c89e13a77401>:104: FutureWarning: Series.__getitem__ treating keys as positions is deprecated

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

def create_spark_session(app_name="TradingAnalysis"):
    """Create and configure Spark session"""
    return SparkSession.builder \
        .appName(app_name) \
        .config("spark.sql.execution.arrow.enabled", "true") \
        .config("spark.driver.memory", "4g") \
        .getOrCreate()

class SparkTechnicalAnalyzer:
    def __init__(self, spark_session):
        self.spark = spark_session

    def calculate_indicators(self, df):
        """Calculate technical indicators using pure Spark operations"""
        # Define windows for different periods
        window_20 = Window.orderBy("Date").rowsBetween(-19, 0)
        window_50 = Window.orderBy("Date").rowsBetween(-49, 0)
        window_14 = Window.orderBy("Date").rowsBetween(-13, 0)
        window_rsi = Window.orderBy("Date").rowsBetween(-14, -1)

        # Calculate price changes for RSI
        df = df.withColumn("price_change", col("Close") - lag("Close", 1).over(Window.orderBy("Date")))

        # Calculate moving averages and basic indicators
        df = df.withColumns({
            "SMA20": avg("Close").over(window_20),
            "SMA50": avg("Close").over(window_50),
            "Volume_MA20": avg("Volume").over(window_20),
            "Volatility": stddev("Close").over(window_20),
            "Upper_Band": avg("Close").over(window_20) + (2 * stddev("Close").over(window_20)),
            "Lower_Band": avg("Close").over(window_20) - (2 * stddev("Close").over(window_20)),
            "Gain": when(col("price_change") > 0, col("price_change")).otherwise(0),
            "Loss": when(col("price_change") < 0, -col("price_change")).otherwise(0)
        })

        # Calculate RSI components
        df = df.withColumns({
            "Avg_Gain": avg("Gain").over(window_14),
            "Avg_Loss": avg("Loss").over(window_14)
        })

        # Calculate final RSI
        df = df.withColumn(
            "RSI",
            100 - (100 / (1 + col("Avg_Gain") / col("Avg_Loss")))
        )

        return df.na.drop()

class SparkTradingSystem:
    def __init__(self, spark_session, symbol, initial_capital=100000):
        self.spark = spark_session
        self.symbol = symbol
        self.initial_capital = initial_capital
        self.analyzer = SparkTechnicalAnalyzer(spark_session)

    def prepare_data(self, start_date, end_date):
        """Fetch and prepare market data"""
        # Get data from yfinance
        ticker = yf.Ticker(self.symbol)
        pdf = ticker.history(start=start_date, end=end_date)

        # Convert to Spark DataFrame
        return self.spark.createDataFrame(pdf.reset_index())

    def generate_signals(self, df):
        """Generate trading signals based on technical analysis"""
        # Calculate technical indicators
        df = self.analyzer.calculate_indicators(df)

        # Create trading signals
        df = df.withColumn("Signal",
            when((col("Close") > col("SMA20")) &
                 (col("SMA20") > col("SMA50")) &
                 (col("RSI") < 70) &
                 (col("Volume") > col("Volume_MA20")), 1)
            .when((col("Close") < col("SMA20")) &
                  (col("SMA20") < col("SMA50")) &
                  (col("RSI") > 30) &
                  (col("Volume") > col("Volume_MA20")), -1)
            .otherwise(0)
        )

        return df

    def backtest(self, signals_df):
        """Execute backtest simulation"""
        # Register DataFrame for SQL operations
        signals_df.createOrReplaceTempView("trading_signals")

        # Calculate portfolio values using Spark SQL
        results = self.spark.sql(f"""
            WITH trades AS (
                SELECT
                    Date,
                    Close,
                    Signal,
                    CASE
                        WHEN Signal = 1 THEN -Close
                        WHEN Signal = -1 THEN Close
                        ELSE 0
                    END as Trade_Value,
                    RSI,
                    Volatility
                FROM trading_signals
                WHERE Signal != 0
            ),
            portfolio AS (
                SELECT
                    Date,
                    Close,
                    Signal,
                    Trade_Value,
                    RSI,
                    Volatility,
                    sum(Trade_Value) OVER (ORDER BY Date) + {self.initial_capital} as Portfolio_Value
                FROM trades
            )
            SELECT *,
                (Portfolio_Value - {self.initial_capital}) / {self.initial_capital} as Returns
            FROM portfolio
            ORDER BY Date
        """)

        return results

    def analyze_performance(self, results_df):
        """Calculate performance metrics"""
        pdf = results_df.toPandas()

        if len(pdf) == 0:
            return {
                "total_returns": 0,
                "sharpe_ratio": 0,
                "max_drawdown": 0,
                "total_trades": 0,
                "win_rate": 0,
                "final_value": self.initial_capital
            }

        returns = pdf['Returns']

        metrics = {
            "total_returns": float(returns.iloc[-1]),
            "sharpe_ratio": float(np.sqrt(252) * returns.mean() / returns.std()) if len(returns) > 1 else 0,
            "max_drawdown": float((pdf['Portfolio_Value'].cummax() - pdf['Portfolio_Value']).max() /
                          pdf['Portfolio_Value'].cummax().max()),
            "total_trades": len(pdf),
            "win_rate": len(pdf[pdf['Returns'] > 0]) / len(pdf) if len(pdf) > 0 else 0,
            "final_value": float(pdf['Portfolio_Value'].iloc[-1])
        }

        return metrics

def main():
    # Initialize Spark
    spark = create_spark_session()

    # Create trading system
    system = SparkTradingSystem(
        spark_session=spark,
        symbol="AAPL",
        initial_capital=100000
    )

    # Execute trading strategy
    data = system.prepare_data("2023-01-01", "2023-12-31")
    signals = system.generate_signals(data)
    results = system.backtest(signals)
    metrics = system.analyze_performance(results)

    # Print performance summary
    print("\nTrading Performance Summary")
    print("=" * 30)
    print(f"Total Return: {metrics['total_returns']:.2%}")
    print(f"Sharpe Ratio: {metrics['sharpe_ratio']:.2f}")
    print(f"Maximum Drawdown: {metrics['max_drawdown']:.2%}")
    print(f"Total Trades: {metrics['total_trades']}")
    print(f"Win Rate: {metrics['win_rate']:.2%}")
    print(f"Final Portfolio Value: ${metrics['final_value']:,.2f}")

    # Stop Spark session
    spark.stop()

if __name__ == "__main__":
    main()


Trading Performance Summary
Total Return: -4.38%
Sharpe Ratio: -35.89
Maximum Drawdown: 4.58%
Total Trades: 49
Win Rate: 0.00%
Final Portfolio Value: $95,621.84


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

def create_spark_session():
    """建立 Spark 連線"""
    return SparkSession.builder \
        .appName("TradingSystem") \
        .config("spark.sql.execution.arrow.enabled", "true") \
        .getOrCreate()

class TradingSystem:
    def __init__(self, spark, symbol, initial_capital=1000000):
        self.spark = spark
        self.symbol = symbol
        self.initial_capital = initial_capital

    def prepare_data(self, start_date, end_date):
        """準備股票數據"""
        ticker = yf.Ticker(self.symbol)
        pdf = ticker.history(start=start_date, end=end_date)
        return self.spark.createDataFrame(pdf.reset_index())

    def calculate_indicators(self, df):
        """計算技術指標"""
        # 定義時間窗口
        window_20 = Window.orderBy("Date").rowsBetween(-19, 0)
        window_50 = Window.orderBy("Date").rowsBetween(-49, 0)

        # 計算技術指標
        df = df.withColumns({
            "SMA20": avg("Close").over(window_20),
            "SMA50": avg("Close").over(window_50),
            "VolMA20": avg("Volume").over(window_20),
            "PriceChange": col("Close") - lag("Close", 1).over(Window.orderBy("Date")),
            "Volatility": stddev("Close").over(window_20)
        })

        # 生成交易信號
        df = df.withColumn("Signal",
            when((col("Close") > col("SMA20")) &
                 (col("SMA20") > col("SMA50")) &
                 (col("Volume") > col("VolMA20")), 1)
            .when((col("Close") < col("SMA20")) &
                  (col("SMA20") < col("SMA50")) &
                  (col("Volume") > col("VolMA20")), -1)
            .otherwise(0)
        )

        return df.na.drop()

    def simulate_trades(self, df):
        """模擬交易"""
        # 選擇需要的列並重命名
        trades_df = df.select(
            col("Date"),
            col("Close"),
            col("Signal"),
            col("Volatility")
        ).filter(col("Signal") != 0)

        # 計算交易金額
        trades_df = trades_df.withColumn(
            "TradeValue",
            when(col("Signal") == 1, -col("Close"))
            .when(col("Signal") == -1, col("Close"))
            .otherwise(0)
        )

        # 計算累積資金變化
        window_spec = Window.orderBy("Date")
        portfolio_df = trades_df.withColumn(
            "PortfolioValue",
            sum("TradeValue").over(window_spec) + lit(self.initial_capital)
        )

        # 計算報酬率
        portfolio_df = portfolio_df.withColumn(
            "Returns",
            (col("PortfolioValue") - lit(self.initial_capital)) / lit(self.initial_capital)
        )

        return portfolio_df

    def analyze_performance(self, portfolio_df):
        """分析交易績效"""
        pdf = portfolio_df.toPandas()

        if len(pdf) == 0:
            return {
                "總報酬率": 0,
                "夏普比率": 0,
                "最大回撤": 0,
                "總交易次數": 0,
                "勝率": 0
            }

        # 計算績效指標
        total_return = float(pdf["Returns"].iloc[-1])
        daily_returns = pdf["PortfolioValue"].pct_change().dropna()
        sharpe = np.sqrt(252) * daily_returns.mean() / daily_returns.std() if len(daily_returns) > 1 else 0

        # 計算最大回撤
        portfolio_values = pdf["PortfolioValue"]
        rolling_max = portfolio_values.cummax()
        drawdowns = (rolling_max - portfolio_values) / rolling_max
        max_drawdown = drawdowns.max()

        return {
            "總報酬率": total_return,
            "夏普比率": float(sharpe),
            "最大回撤": float(max_drawdown),
            "總交易次數": len(pdf),
            "勝率": len(pdf[pdf["Returns"] > 0]) / len(pdf) if len(pdf) > 0 else 0
        }

def main():
    # 初始化 Spark
    spark = create_spark_session()

    # 創建交易系統實例
    trading_system = TradingSystem(
        spark=spark,
        symbol="2330.TW",
        initial_capital=1000000
    )

    # 執行回測
    print("開始回測交易策略...")

    # 準備數據
    df = trading_system.prepare_data("2023-01-01", "2023-12-31")

    # 計算指標和信號
    df_with_signals = trading_system.calculate_indicators(df)

    # 模擬交易
    portfolio_results = trading_system.simulate_trades(df_with_signals)

    # 分析績效
    performance = trading_system.analyze_performance(portfolio_results)

    # 輸出結果
    print("\n交易績效分析")
    print("================")
    print(f"總報酬率: {performance['總報酬率']:.2%}")
    print(f"夏普比率: {performance['夏普比率']:.2f}")
    print(f"最大回撤: {performance['最大回撤']:.2%}")
    print(f"總交易次數: {performance['總交易次數']}")
    print(f"勝率: {performance['勝率']:.2%}")

    # 關閉 Spark 連線
    spark.stop()

if __name__ == "__main__":
    main()

開始回測交易策略...

交易績效分析
總報酬率: -0.79%
夏普比率: -4.48
最大回撤: 0.96%
總交易次數: 51
勝率: 13.73%


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime

def create_spark_session():
    """建立 Spark 連線"""
    return SparkSession.builder \
        .appName("StockAnalysis") \
        .config("spark.sql.execution.arrow.enabled", "true") \
        .getOrCreate()

class 股票分析系統:
    def __init__(self, spark, 股票代號, 初始資金=1000000):
        self.spark = spark
        self.股票代號 = 股票代號
        self.初始資金 = 初始資金

    def 取得股票數據(self, 開始日期, 結束日期):
        """下載並準備股票數據"""
        # 使用 yfinance 下載數據
        ticker = yf.Ticker(self.股票代號)
        df = ticker.history(start=開始日期, end=結束日期)

        # 重新命名欄位為中文
        df = df.rename(columns={
            'Open': '開盤價',
            'High': '最高價',
            'Low': '最低價',
            'Close': '收盤價',
            'Volume': '成交量'
        })

        # 轉換為 Spark DataFrame
        return self.spark.createDataFrame(df.reset_index())

    def 計算技術指標(self, df):
        """計算技術指標"""
        # 定義時間窗口
        window_20 = Window.orderBy("Date").rowsBetween(-19, 0)
        window_50 = Window.orderBy("Date").rowsBetween(-49, 0)

        # 計算技術指標
        df = df.withColumns({
            "MA20": avg("收盤價").over(window_20),
            "MA50": avg("收盤價").over(window_50),
            "成交量MA20": avg("成交量").over(window_20),
            "日報酬率": ((col("收盤價") - lag("收盤價", 1).over(Window.orderBy("Date"))) / lag("收盤價", 1).over(Window.orderBy("Date"))),
            "波動率": stddev("收盤價").over(window_20)
        })

        return df.na.drop()

    def 生成交易訊號(self, df):
        """生成交易訊號"""
        return df.withColumn("交易訊號",
            when((col("收盤價") > col("MA20")) &
                 (col("MA20") > col("MA50")) &
                 (col("成交量") > col("成交量MA20")), 1)
            .when((col("收盤價") < col("MA20")) &
                  (col("MA20") < col("MA50")) &
                  (col("成交量") > col("成交量MA20")), -1)
            .otherwise(0)
        )

    def 模擬交易(self, df):
        """執行交易模擬"""
        # 選擇需要的欄位
        交易df = df.select(
            col("Date"),
            col("收盤價"),
            col("交易訊號"),
            col("波動率")
        ).filter(col("交易訊號") != 0)

        # 計算交易損益
        交易df = 交易df.withColumn(
            "交易金額",
            when(col("交易訊號") == 1, -col("收盤價"))
            .when(col("交易訊號") == -1, col("收盤價"))
            .otherwise(0)
        )

        # 計算投資組合價值
        window_spec = Window.orderBy("Date")
        return 交易df.withColumn(
            "投資組合價值",
            sum("交易金額").over(window_spec) + lit(self.初始資金)
        )

def 主程式():
    # 初始化 Spark
    spark = create_spark_session()
    print("Spark 連線已建立")

    # 建立分析系統
    分析系統 = 股票分析系統(
        spark=spark,
        股票代號="2330.TW",
        初始資金=1000000
    )

    try:
        # 取得股票數據
        print("開始下載股票數據...")
        df = 分析系統.取得股票數據("2023-01-01", "2023-12-31")
        print("股票數據下載完成")

        # 計算指標
        print("計算技術指標...")
        df = 分析系統.計算技術指標(df)

        # 生成訊號
        print("生成交易訊號...")
        df = 分析系統.生成交易訊號(df)

        # 模擬交易
        print("執行交易模擬...")
        結果 = 分析系統.模擬交易(df)

        # 顯示結果
        print("\n交易結果分析:")
        結果.show(5)

    except Exception as e:
        print(f"執行過程發生錯誤: {str(e)}")
    finally:
        # 關閉 Spark 連線
        spark.stop()
        print("Spark 連線已關閉")

if __name__ == "__main__":
    主程式()

Spark 連線已建立
開始下載股票數據...
股票數據下載完成
計算技術指標...
生成交易訊號...
執行交易模擬...

交易結果分析:
+-------------------+-----------------+--------+------------------+------------------+-----------------+
|               Date|           收盤價|交易訊號|            波動率|          交易金額|     投資組合價值|
+-------------------+-----------------+--------+------------------+------------------+-----------------+
|2023-02-14 16:00:00|507.5482177734375|       1|22.232459210597575|-507.5482177734375|999492.4517822266|
|2023-03-16 16:00:00|503.4902648925781|       1| 5.424755958110751|-503.4902648925781| 998988.961517334|
|2023-03-21 16:00:00|518.0701293945312|       1| 6.443167279167709|-518.0701293945312|998470.8913879395|
|2023-03-22 16:00:00|  522.93017578125|       1| 8.205440756048082|  -522.93017578125|997947.9612121582|
|2023-03-30 16:00:00|518.0701293945312|       1|10.449343237543305|-518.0701293945312|997429.8910827637|
+-------------------+-----------------+--------+------------------+------------------+-----------------+
onl

In [ ]:
from dataclasses import dataclass
from typing import Dict, List
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

@dataclass
class 交易區間:
    下限: float
    上限: float
    停損點: float
    目標獲利: float

class 交易規則系統:
    def __init__(self, spark_session, 股票代碼: str):
        self.spark = spark_session
        self.股票代碼 = 股票代碼
        self.區間數量 = 8
        self.持有期間 = 15
        self.目標獲利 = 0.0592
        self.進場門檻 = 0.8918

    def 計算價格區間(self, 數據: pd.DataFrame) -> List[交易區間]:
        """計算價格交易區間"""
        最低價 = 數據.agg(min("最低價")).collect()[0][0]
        最高價 = 數據.agg(max("最高價")).collect()[0][0]
        區間長度 = (最高價 - 最低價) / self.區間數量

        交易區間列表 = []
        for i in range(self.區間數量):
            下限 = 最低價 + i * 區間長度
            上限 = 最低價 + (i + 1) * 區間長度

            區間 = 交易區間(
                下限=下限,
                上限=上限,
                停損點=下限,
                目標獲利=self.目標獲利
            )
            交易區間列表.append(區間)

        return 交易區間列表

    def 檢查趨勢確認(self, 數據: pd.DataFrame) -> pd.DataFrame:
        """檢查趨勢確認條件"""
        window_20 = Window.orderBy("Date").rowsBetween(-19, 0)
        window_60 = Window.orderBy("Date").rowsBetween(-59, 0)

        return 數據.withColumns({
            "SMA20": avg("收盤價").over(window_20),
            "SMA60": avg("收盤價").over(window_60),
            "趨勢確認": when(
                (col("SMA20") > col("SMA60")) &
                (col("收盤價") > col("SMA20")),
                lit(True)
            ).otherwise(lit(False))
        })

    def 檢查技術指標(self, 數據: pd.DataFrame) -> pd.DataFrame:
        """檢查技術指標條件"""
        window_14 = Window.orderBy("Date").rowsBetween(-13, 0)

        # 計算RSI
        數據 = 數據.withColumn(
            "價格變化",
            col("收盤價") - lag("收盤價", 1).over(Window.orderBy("Date"))
        ).withColumns({
            "上漲": when(col("價格變化") > 0, col("價格變化")).otherwise(0),
            "下跌": when(col("價格變化") < 0, -col("價格變化")).otherwise(0)
        })

        數據 = 數據.withColumns({
            "RSI": 100 - (100 / (1 +
                avg("上漲").over(window_14) /
                avg("下跌").over(window_14)
            )),
            "RSI_確認": (col("RSI") > 30) & (col("RSI") < 70)
        })

        return 數據

    def 檢查成交量(self, 數據: pd.DataFrame) -> pd.DataFrame:
        """檢查成交量條件"""
        window_20 = Window.orderBy("Date").rowsBetween(-19, 0)

        return 數據.withColumns({
            "成交量均線": avg("成交量").over(window_20),
            "成交量確認": col("成交量") > col("成交量均線")
        })

    def 生成交易訊號(self, 數據: pd.DataFrame, 區間: 交易區間) -> pd.DataFrame:
        """生成交易訊號"""
        return 數據.withColumn(
            "交易訊號",
            when(
                (col("收盤價").between(區間.下限, 區間.上限)) &
                (col("趨勢確認")) &
                (col("RSI_確認")) &
                (col("成交量確認")),
                lit(1)
            ).otherwise(lit(0))
        )

    def 執行風險管理(self, 數據: pd.DataFrame) -> pd.DataFrame:
        """執行風險管理檢查"""
        window_20 = Window.orderBy("Date").rowsBetween(-19, 0)

        # 計算波動率
        數據 = 數據.withColumn(
            "波動率",
            stddev("收盤價").over(window_20) / avg("收盤價").over(window_20)
        )

        # 價格位置檢查
        數據 = 數據.withColumns({
            "近期最高": max("最高價").over(window_20),
            "近期最低": min("最低價").over(window_20),
            "價格位置": (col("收盤價") - col("近期最低")) /
                       (col("近期最高") - col("近期最低"))
        })

        # 綜合風險確認
        數據 = 數據.withColumn(
            "風險確認",
            (col("波動率") < 0.02) &
            (col("價格位置").between(0.3, 0.7))
        )

        return 數據

    def 產生交易規則(self) -> str:
        規則說明 = f"""
交易系統規則說明
================

一、基本參數設定：
- 區間數量：{self.區間數量}個
- 持有期間：{self.持有期間}天
- 目標獲利：{self.目標獲利:.2%}
- 進場門檻：{self.進場門檻:.2%}

二、進場條件：
1. 趨勢確認
   - 20日均線在60日均線之上
   - 收盤價在20日均線之上

2. 技術指標確認
   - RSI介於30到70之間
   - 成交量大於20日均量

3. 風險控制
   - 波動率小於2%
   - 價格位置在近期高低點30%-70%之間
   - 嚴格執行停損點設置

三、出場條件：
1. 持有期間到期（{self.持有期間}天）
2. 達到目標獲利（{self.目標獲利:.2%}）
3. 觸及停損點
4. 技術指標反轉

四、部位管理：
1. 單一部位不超過總資金的5%
2. 總部位不超過總資金的50%
3. 相同區間不重複建倉

五、風險管理：
1. 嚴格執行停損機制
2. 定期評估持倉表現
3. 動態調整交易參數
"""
        return 規則說明

def 主程式():
    # 創建Spark連線
    spark = SparkSession.builder \
        .appName("TradingRules") \
        .getOrCreate()

    # 初始化交易系統
    交易系統 = 交易規則系統(spark, "2330.TW")

    # 輸出交易規則
    print(交易系統.產生交易規則())

    spark.stop()

if __name__ == "__main__":
    主程式()


交易系統規則說明

一、基本參數設定：
- 區間數量：8個
- 持有期間：15天
- 目標獲利：5.92%
- 進場門檻：89.18%

二、進場條件：
1. 趨勢確認
   - 20日均線在60日均線之上
   - 收盤價在20日均線之上

2. 技術指標確認
   - RSI介於30到70之間
   - 成交量大於20日均量

3. 風險控制
   - 波動率小於2%
   - 價格位置在近期高低點30%-70%之間
   - 嚴格執行停損點設置

三、出場條件：
1. 持有期間到期（15天）
2. 達到目標獲利（5.92%）
3. 觸及停損點
4. 技術指標反轉

四、部位管理：
1. 單一部位不超過總資金的5%
2. 總部位不超過總資金的50%
3. 相同區間不重複建倉

五、風險管理：
1. 嚴格執行停損機制
2. 定期評估持倉表現
3. 動態調整交易參數

